In [26]:
import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 750)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.2276e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 2000


In [3]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1 
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("../../../data/gps.dada")
print(dada.get_header())
dada = dada.combined_data()



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [81]:
array.steering_vector([0,0], wavelength)

array([ 0.28936544-0.9572187j , -0.38367628+0.92346765j,
       -0.8083881 -0.58864988j,  0.89296643+0.45012327j,
        0.63408678+0.77326189j, -0.54450583-0.83875706j,
       -0.71451342+0.69962174j,  0.99988422-0.01521692j,
        0.12862974+0.99169269j, -0.99080042+0.13533121j,
        0.3375557 +0.94130556j, -0.91359294+0.40662998j,
        0.22982853+0.97323114j, -0.9492535 +0.31451198j,
        0.37860534-0.92555821j,  0.70924556+0.70496152j,
        0.77494962-0.63202301j, -0.48599853+0.87395963j,
        0.99237859-0.12322637j,  0.06867479-0.9976391j ,
       -0.94016331-0.34072417j,  0.07684229-0.99704326j,
        0.67667242-0.73628421j,  0.94554284+0.32549769j,
        0.53298596+0.84612409j,  0.01641594+0.99986525j,
        0.77013833+0.63787691j, -0.99937132+0.0354537j ,
       -0.11375228-0.99350914j, -0.98203582+0.1886946j ,
        0.32209399+0.94670769j, -0.86295217+0.50528561j,
        0.50018058-0.86592112j, -0.07006181+0.99754265j,
        0.65739175-0.753549j  ,

In [4]:
dada.shape

(208896, 57, 64, 2)

In [5]:
dada = dada[0:2000, core_indices, GPS_CHANNEL, POL].T
X = dada

In [6]:
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_9853/1887256366.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp)


datetime.datetime(2024, 2, 16, 11, 21, 22, 92702)

In [156]:
array_factor = array.steering_vector([220.67, 65.97], wavelength)
#X = X.T
X_adj = X.copy()
for i, shift in enumerate(array_factor):
    X_adj[i, :] = X_adj[i, :] * array_factor[i].conj()

R = X.conj().T @ X / N
R_adj = X_adj.conj().T @ X_adj / N

In [145]:
import numpy as np

evals, evecs = np.linalg.eigh(R)
R

array([[43939.        +0.j     , -1425.6279 +5452.279j  ,
         3486.9534 -3225.442j  , ...,  2013.7675 -5584.279j  ,
        -1126.8837 +4012.721j  ,  3578.5813  +365.7907j ],
       [-1425.6279 -5452.279j  , 41073.07      +0.j     ,
        -3321.558  -1750.2325j , ..., -2459.535   +897.32556j,
         4608.186  -3454.j     , -1240.0697 -4017.535j  ],
       [ 3486.9534 +3225.442j  , -3321.558  +1750.2325j ,
        45996.188     +0.j     , ...,  7377.93   +2624.1628j ,
        -3862.3489 +2576.093j  , -1446.4884 +4791.93j   ],
       ...,
       [ 2013.7675 +5584.279j  , -2459.535   -897.32556j,
         7377.93   -2624.1628j , ..., 46125.605     +0.j     ,
        -5846.442   +347.4186j ,  -841.48834+6785.7207j ],
       [-1126.8837 -4012.721j  ,  4608.186  +3454.j     ,
        -3862.3489 -2576.093j  , ..., -5846.442   -347.4186j ,
        43681.348     +0.j     ,  1308.0465 -5801.1396j ],
       [ 3578.5813  -365.7907j , -1240.0697 +4017.535j  ,
        -1446.4884 -4791.93j  

In [146]:
R_adj

array([[ 904.3023  +1.62473207e-05j,   21.397081-1.05773758e+02j,
         -79.75975 -5.05234413e+01j, ..., -253.46132 -1.31544952e+01j,
          83.55547 +2.07944443e+02j,   24.074514-2.81047668e+02j],
       [  21.397081+1.05773766e+02j,  906.1395  +1.84472094e-06j,
         -21.01408 +1.37634125e+02j, ...,   70.927315-1.72255341e+02j,
        -386.4405  +2.42494186e+02j,  -29.720984-3.21199341e+02j],
       [ -79.75975 +5.05234528e+01j,  -21.01408 -1.37634125e+02j,
         925.1628  -4.43094723e-06j, ...,   44.526066-1.90868408e+02j,
          48.791862+4.38486542e+02j, -215.87999 +1.53416473e+02j],
       ...,
       [-253.46132 +1.31544838e+01j,   70.927315+1.72255341e+02j,
          44.526066+1.90868408e+02j, ...,  818.76746 +0.00000000e+00j,
        -290.27908 -3.06069763e+02j,   94.46512 +1.34186050e+02j],
       [  83.55547 -2.07944427e+02j, -386.4405  -2.42494110e+02j,
          48.791862-4.38486694e+02j, ..., -290.27908 +3.06069763e+02j,
        1254.8605  +0.00000000e+00j

In [125]:
import matplotlib.pyplot as plt

plt.plot(evals)
plt.show()

2025-06-03 17:35:09.741 Python[9853:18965885] _TIPropertyValueIsValid called with 16 on nil context!
2025-06-03 17:35:09.741 Python[9853:18965885] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-06-03 17:35:09.741 Python[9853:18965885] Text input context does not respond to _valueForTIProperty:
2025-06-03 17:35:11.885 Python[9853:18965885] _TIPropertyValueIsValid called with 16 on nil context!
2025-06-03 17:35:11.885 Python[9853:18965885] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-06-03 17:35:11.885 Python[9853:18965885] Text input context does not respond to _valueForTIProperty:


In [57]:
evals[-15:]

array([ 36450.5  ,  37099.656,  37439.64 ,  37747.164,  38575.91 ,
        39540.45 ,  40307.95 ,  41162.973,  42905.89 ,  44738.445,
        48184.36 ,  56458.53 , 106063.61 , 139550.8  , 296141.47 ],
      dtype=float32)

In [157]:


theta_steps = 1_000
phi_steps = 1_000

output = direction_of_arrival.MUSICDOA2D().get_directions(
    array,
    R_adj, 
    num_interferers=5,
    wavelength = wavelength,
    theta_min_deg = 0,
    theta_max_deg = 90,
    theta_steps=theta_steps,
    phi_min_deg = -180,
    phi_max_deg = 180,
    phi_steps=phi_steps,
)

In [158]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.999975)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

The MJD I want is 60356.47317236924195




In [159]:
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json
ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')

Loaded 54 satellites


In [160]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u

# Observer location (MeerKAT)
location = EarthLocation(lat=-30.71106*u.deg,
                         lon=21.44389*u.deg,
                         height=1086.6*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print("Source RA/DEC")
print(f"Altitude: {altaz.alt:.2f}")
print(f"Azimuth: {altaz.az:.2f}")

Source RA/DEC
Altitude: 24.03 deg
Azimuth: 229.33 deg


In [161]:
from skyfield.api import wgs84, Topos

meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site = meerkat
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    # any of them should give essentially the same result.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt.degrees
    event_dict['az'] = az.degrees
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
print(len(sats_visible))
df.with_columns(theta=pl.lit(90) - pl.col('alt'), phi=(pl.lit(90) - pl.col('az'))).sort(by="alt", descending=False)


10


name,alt,az,theta,phi
str,f64,f64,f64,f64
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,85.903507,-51.102433
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,81.951034,44.98524
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,73.357624,-1.633704
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,67.443908,-155.454394
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,53.142185,69.596966
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,52.019064,-219.60797
"""NAVSTAR 64 (USA 206)""",44.369909,123.403165,45.630091,-33.403165
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,44.627335,-129.380969
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,40.096014,-50.065598


In [162]:
output.with_columns(dist_from_theta=90 - pl.col('theta'), dist_from_phi = (90-pl.col('phi')) % 360).sort(by="Q", descending=True).head(10)

phi,theta,Q,dist_from_theta,dist_from_phi
f64,f64,f64,f64,f64
88.468468,56.756757,0.0,33.243243,1.531532
-163.423423,5.225225,-0.339769,84.774775,253.423423
35.135135,89.279279,-0.610294,0.720721,54.864865
98.558559,5.945946,-0.620601,84.054054,351.441441
35.135135,89.189189,-0.630185,0.810811,54.864865
-123.063063,38.198198,-0.645726,51.801802,213.063063
35.135135,89.369369,-0.652113,0.630631,54.864865
-110.810811,49.369369,-0.65797,40.630631,200.810811
-142.882883,36.036036,-0.674254,53.963964,232.882883


In [163]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


In [164]:
## Near-field limit calculation
1500**2 / (constants.c / 1.276e9)

9576625.173138944

In [165]:
phi_deg = -118.7
theta_deg = 109.01

boresight_az = 229.33
boresight_alt = 24.04

def spherical_to_cartesian(phi_deg, theta_deg):
    phi = np.radians(phi_deg)
    theta = np.radians(theta_deg)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return np.array([x, y, z])  # In array's frame

def az_alt_to_enu(az_deg, alt_deg):
    az = np.radians(az_deg)
    alt = np.radians(alt_deg)
    e = np.cos(alt) * np.sin(az)
    n = np.cos(alt) * np.cos(az)
    u = np.sin(alt)
    return np.array([e, n, u])

def get_rotation_matrix_from_boresight(az_deg, alt_deg):
    # z-axis: boresight direction
    z_enu = az_alt_to_enu(az_deg, alt_deg)
    z_enu = z_enu / np.linalg.norm(z_enu)

    # Temporary up vector (in ENU): global "Up" is [0, 0, 1]
    up = np.array([0, 0, 1])

    # x-axis: perpendicular to z and up (or use [1, 0, 0] if z == up)
    x_enu = np.cross(up, z_enu)
    if np.linalg.norm(x_enu) < 1e-6:  # if pointing straight up
        x_enu = np.array([1, 0, 0])
    x_enu = x_enu / np.linalg.norm(x_enu)

    # y-axis: completes right-handed system
    y_enu = np.cross(z_enu, x_enu)

    # Construct rotation matrix: columns are [x, y, z]
    R = np.column_stack((x_enu, y_enu, z_enu))
    return R

# MUSIC direction (phi, theta) in array frame
vec_music = spherical_to_cartesian(phi_deg, theta_deg)  # array frame

# Get rotation matrix based on array pointing
R = get_rotation_matrix_from_boresight(boresight_az, boresight_alt)

# Rotate into ENU
vec_enu = R @ vec_music


df = df.with_columns(
    pl.struct(["az", "alt"]).map_elements(lambda row: az_alt_to_enu(row["az"], row["alt"])).alias("result")
)

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_9853/270478294.py:54: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



In [166]:
output.with_columns(
    pl.struct(['phi', 'theta']).map_elements(lambda row: spherical_to_cartesian(row['phi'], row['theta'])).alias('result')
).sort(by='Q', descending=True).head(10)

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_9853/2442223847.py:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



phi,theta,Q,result
f64,f64,f64,object
88.468468,56.756757,0.0,[0.02235322 0.83605204 0.5481946 ]
-163.423423,5.225225,-0.339769,[-0.08728605 -0.02598225 0.9958444 ]
35.135135,89.279279,-0.610294,[0.81773226 0.57546132 0.01257862]
98.558559,5.945946,-0.620601,[-0.0154163 0.10243661 0.99462007]
35.135135,89.189189,-0.630185,[0.81771507 0.57544923 0.01415085]
-123.063063,38.198198,-0.645726,[-0.33736651 -0.51824918 0.78587634]
35.135135,89.369369,-0.652113,[0.81774742 0.57547199 0.01100636]
-110.810811,49.369369,-0.65797,[-0.2696328 -0.70941012 0.65118003]
-142.882883,36.036036,-0.674254,[-0.46910778 -0.35500378 0.80864715]


In [167]:
df

name,alt,az,result
str,f64,f64,object
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,[ 0.95772156 -0.0273154 0.28639706]
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,[0.70032124 0.69996053 0.14001934]
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,[ 0.41343533 -0.49386023 0.76496621]
"""NAVSTAR 60 (USA 196)""",76.956355,197.642354,[-0.06840184 -0.21507817 0.97419842]
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,[-0.60726064 0.50251198 0.61539924]
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,[0.27894145 0.7499294 0.59983128]
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,[-0.4457132 -0.54298774 0.71169098]
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,[-0.84004818 -0.38363983 0.38358772]
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,[ 0.62632575 -0.77628146 0.07143639]


In [69]:
R @ np.array([0,0,1])

array([-0.69268646, -0.59517361,  0.40737432])

In [70]:
az_alt_to_enu(boresight_az, boresight_alt)

array([-0.69268646, -0.59517361,  0.40737432])

In [75]:
def build_rotation_matrix_aligned_with_east(signal_dir):
    z_hat = signal_dir / np.linalg.norm(signal_dir)
    east = np.array([1.0, 0.0, 0.0])

    x_proj = east - np.dot(east, z_hat) * z_hat
    if np.linalg.norm(x_proj) < 1e-6:
        # fallback to north if east is colinear with signal direction
        north = np.array([0.0, 1.0, 0.0])
        x_proj = north - np.dot(north, z_hat) * z_hat

    x_hat = x_proj / np.linalg.norm(x_proj)
    y_hat = np.cross(z_hat, x_hat)

    R = np.column_stack((x_hat, y_hat, z_hat))  # signal → ENU
    return R

R2 = build_rotation_matrix_aligned_with_east(np.array([-0.69268646, -0.59517361, 0.40737432]))
R2 @ np.array([1,0,0])

array([ 0.72123884, -0.5716119 ,  0.3912472 ])

In [109]:
output.with_columns(
    pl.struct(['phi', 'theta']).map_elements(lambda row: spherical_to_cartesian(row['phi'], row['theta'])).alias('result')
).sort(by='Q', descending=True).head(10)

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_9853/2442223847.py:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



phi,theta,Q,result
f64,f64,f64,object
-43.063063,50.36036,0.0,[ 0.56261666 -0.52580745 0.63795691]
95.315315,51.801802,-0.008754,[-0.07280103 0.78249705 0.61838368]
-172.792793,52.432432,-0.178836,[-0.78637223 -0.09944241 0.60969659]
34.774775,8.018018,-0.191212,[0.11457263 0.07955527 0.99022425]
77.297297,81.171171,-0.333999,[0.21728678 0.96396546 0.15348305]
-93.873874,27.747748,-0.351503,[-0.03145473 -0.46451597 0.88500594]
12.792793,87.657658,-0.360098,[0.97436242 0.22124082 0.0408702 ]
58.918919,25.315315,-0.364609,[0.22074849 0.3662123 0.90396828]
12.792793,87.567568,-0.366912,[0.97429855 0.22122632 0.0424412 ]


In [83]:
(90 - 229.33) % 360

220.67

In [85]:
90 - 24.03

65.97

In [88]:
np.sin(np.deg2rad(65.97)) * np.sin(np.deg2rad(220.67))

np.float64(-0.5952199413798546)